# buy and hold 전력
1. 구매 후 일정 기간이 지나면 판매

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../csv/AAPL.csv", index_col='Date')
# 인덱스를 변경
# df = df.set_index('Date')
df.head(3)

In [ ]:
# 결측치를 확인 
df.info()

In [ ]:
# volume 데이터의 결측치인 데이터를 출력
# 데이터프레임명.loc[인덱스의 조건식, 컬럼의 조건식]
df.loc[df['Volume'].isna()]

In [ ]:
# volumn 데이터의 결측치가 아닌 데이터를 출력
df.loc[~df['Volume'].isna()]

In [ ]:
df2 = df.copy()

In [ ]:
df2.dropna(axis=0)

In [ ]:
df2.drop(["1981-08-10", '1981-08-21'], axis=0)

In [ ]:
# 결측치와 양의 무한대 음의 무한대의 값들을 제거 
df.loc[df['Close'] != np.inf]
df.loc[df['Close'] != -np.inf]
df.loc[~df['Close'].isna()]

In [ ]:
df.loc[
    (df['Close'] != np.inf) & (df['Close'] != -np.inf) & (~df['Close'].isna())
]

In [ ]:
flag = ~df.isin([np.nan, -np.inf, np.inf]).any(axis=1)

In [ ]:
df = df.loc[flag]

In [ ]:
# df에서 Close를 제외한 나머지 컬럼들을 모두 제거 
price_df = df[['Close']]

In [ ]:
# 일별 수익율??? (오늘의 종가 - 전날의 종가) / 전날의 종가

# case1 (shift())
(price_df['Close'] - price_df['Close'].shift(1)) / price_df['Close'].shift(1)

In [ ]:
# case2 (diff())
price_df['Close'].diff(1) / price_df['Close'].shift(1)

In [ ]:
# case3 (pct_change())
price_df['daily_rtn'] = price_df['Close'].pct_change().fillna(0)

In [ ]:
price_df.head(3)

In [ ]:
# 일별 수익율을 기준으로 총 수익율 계산
test_df = price_df.iloc[2000:5000]

In [ ]:
# 수익율을 계산
rtn = 1

for i in test_df['daily_rtn'].tolist():
    rtn = rtn * (1 + i)

rtn

In [ ]:
# buyandhold 함수화
def bnh(_df, _col, _start, _end):
    # df에 columns에 Date가 존재한다면 Date를 인덱스로 변환
    if 'Date' in _df.columns:
        _df = _df.set_index('Date')
    # index를 시계열 데이터로 변경 (to_datetime(Series데이터, format=Series데이터의 폼))
    _df.index = pd.to_datetime(_df.index, format='%Y-%m-%d')
    # 결측치, 무한대, 음의 무한대 값을 제외시킨다. 
    _df = _df.loc[~_df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]
    # 기준이 되는 컬럼을 제외하고 나머지 컬럼을 제거 
    _df = _df[[_col]]
    # 일별 수익율 계산해서 새로운 파생변수에 대입
    _df["daily_rtn"] = _df[_col].pct_change()
    # 구매한 날과 판매한 날을 기준으로 데이터를 필터링
    _df = _df.loc[_start : _end]
    # 누적 수익율을 계산하여 새로운 파생변수에 대입 
    _df['st_rtn'] = (1 + _df['daily_rtn']).cumprod()

    return _df

In [ ]:
from datetime import datetime

In [ ]:
df = pd.read_csv("../csv/AMZN.csv")
df.tail(1)

In [ ]:
start = '20000101'
end = '20151231'
# 문자열을 시계열로 변경
start = datetime.strptime(start, '%Y%m%d').isoformat()
end = datetime.strptime(end, '%Y%m%d').isoformat()

In [ ]:
bnh(df, 'Adj Close', start, end)

# bollinger band
1. 데이터를 로드 
2. 결측치 이상치 제거 
3. 시간, 종가로 이루어진 데이터프레임을 생성 
4. 이동 평균선 : 데이터의 20개의 평균을 구하여 새로운 파생변수 대입
5. 상단 밴드 : 이동평균선 + (2 * 데이터 20개의 표준편차)
6. 하단 밴드 : 이동평균선 - (2 * 데이터 20개의 표준편차)
7. 구매 상태를 확인하는 파생변수 생성
8. 구매 상태를 입력
9. 수익율 계산

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../csv/AAPL.csv', index_col='Date')

In [ ]:
# 결측치 이상치를 제거 
flag = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
df = df.loc[flag]
df.head(3)

In [ ]:
# 수정종가로 이루어진 데이터프레임 생성
price_df = df[['Adj Close']]
price_df

In [ ]:
# 이동 평균선 생성 
# 종가 데이터의 20개의 평균 값
price_df['center'] = 0
# case1 (반복문)
for i in range(20, len(price_df), 1):
    # i? 인덱스의 위치
    center_data = price_df.iloc[i-20:i-1, 0].mean()
    price_df.iloc[i-1, 1] =center_data
price_df.iloc[19:25]

In [ ]:
# case2 (rolling(n)) : n개 만큼 데이터를 묶는다.
price_df['center'] = price_df['Adj Close'].rolling(20).mean()
price_df.iloc[19:25]

In [ ]:
# 상단 밴드 
price_df['ub'] = price_df['center'] + ( 2 * price_df['Adj Close'].rolling(20).std() )
# 하단 밴드 
price_df['lb'] = price_df['center'] - ( 2 * price_df['Adj Close'].rolling(20).std() )

In [ ]:
price_df.tail(10)

In [ ]:
price_df.tail(100).plot()

In [ ]:
# 거래 내역 파생변수 생성 
price_df['trade'] = ''

In [ ]:
price_df.head(3)

In [ ]:
# 일정 기간으로 인덱스 필터링 
start = '20100101'
start = datetime.strptime(start, '%Y%m%d').isoformat()

price_df = price_df.loc[start:]

### 구매 내역 추가 
- 조건
    - 상단밴드보다 종가가 높은 경우
        - 현재 구매 상태라면
            - 매도(trade = '')
        - 구매 상태가 아니라면
            - 아무 행동도 하지 않는다(trade = '')
    - 하단밴드보다 종가가 낮은 경우
        - 현재 구매 상태라면
            - 아무 행동도 하지 않는다(trade = 'buy')
        - 구매 상태가 아니라면
            - 매수 (trade = 'buy')
    - 밴드 사이에 종가가 존재하는 경우
        - 현재 구매 상태라면 
            - 아무 행동도 하지 않는다(trade = 'buy')
        - 구매 상태가 아니라면
            - 아무 행동도 하지 않는다(trade = '')

In [ ]:
for i in price_df.index:
    # i? -> price_df의 index 값들이 하나씩 대입

    # 상단밴드보다 종가가 높은 경우
    if price_df.loc[i, 'Adj Close'] > price_df.loc[i, 'ub']:
        price_df.loc[i, 'trade'] = ''
    # 하단밴드보다 종가가 낮은 경우
    elif price_df.loc[i, 'Adj Close'] < price_df.loc[i, 'lb']:
        price_df.loc[i, 'trade'] = 'buy'
    # 밴드 사이에 종가가 존재하는 경우
    else:
        # 현재 구매 상태라면? 전날의 trade가 buy인 경우
        if price_df.shift(1).loc[i, 'trade'] == 'buy':
            price_df.loc[i, 'trade'] = 'buy'
        else :
            price_df.loc[i, 'trade'] = ''

In [ ]:
price_df['trade'].value_counts()

## 수익율 계산
- 구매한 날의 종가 
    - 전 날의 trade 데이터가 '' 오늘의 trade가 'buy'인 경우의 Adj Close의 값
- 판매한 날의 종가
    - 전 날의 trade 데이터가 'buy' 오늘의 trade가 ''인 경우의 Adj Close의 값
- 수익율 계산
    - 판매한 날의 종가 / 구매한 날의 종가

In [ ]:
rtn = 1.0
buy = 0.0
sell = 0.0

# 수익율 파생변수 생성
price_df['return'] = 1.0

for i in price_df.index:
    # i? price_df의 index값들이 하나씩 대입하여 실행

    # 구매가
    if (price_df.shift(1).loc[i, 'trade'] == '') & (price_df.loc[i, 'trade'] == 'buy'):
        # 해당하는 날짜의 종가를 buy에 대입
        buy = price_df.loc[i, 'Adj Close']
        print('매수한 날짜 :', i, '매수가 :', buy)
    # 판매가
    elif (price_df.shift(1).loc[i, 'trade'] == 'buy') & (price_df.loc[i, 'trade'] == ''):
        # 판매가를 sell에 대입
        sell = price_df.loc[i, 'Adj Close']
        # 수익율 계산
        rtn = sell / buy
        # 수익율을 해당하는 날짜의 return에 대입
        price_df.loc[i, 'return'] = rtn
        print('매도 날짜 :', i, '매도가 :', sell, '수익율 :', rtn)

In [59]:
# 누적 수익율

acc_rtn = 1.0

for i in price_df.index:
    rtn = price_df.loc[i, 'return']
    acc_rtn *= rtn
    price_df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

C:\Users\EZEN\AppData\Local\Temp\ipykernel_6644\2481751609.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df.loc[i, 'acc_rtn'] = acc_rtn


누적 수익율 : 1.3923287814461949
